# Prototyping notebook for future version

In [36]:
:dep astrors
:dep polars
:dep numpy
:dep plotters
:dep glob
:dep regex
:dep ndarray

In [48]:
use std::path::Path;
use std::fs::File;
use glob::glob;
use polars::prelude::*;
use regex::Regex;
use astrors::io;
use ndarray::Array2;

// Formats for Data that could be found in the experiment data directory
#[derive(Debug)]
enum DataFormat {
    Fits(Fits),
    Csv(Csv),
    Temp(Temp),
}

// Polarization of the data
#[derive(Debug)]
enum Pol {
    S,
    P,
    N,
}

// Structures to hold the data
#[derive(Debug)]
struct DataFile {
    id: SampleID,
    data: Option<DataFormat>,
}

#[derive(Debug)]
struct SampleID {
    name: String,
    scan_id: u32,
    file_id: u32,
}

#[derive(Debug)]
struct Fits {
    pol: Pol,
    energy: f64,
    current: f64,
    hos: f64,
    hes: f64,
    exposure: f64,
    theta: f64,
    image: io::hdus::image::image::ImageData,
}

#[derive(Debug)]
struct Csv {}

#[derive(Debug)]
struct Temp {}

// Implementations for these data structures
impl DataFile {
    fn new(id: SampleID, data: Option<DataFormat>) -> Self {
        DataFile { id, data }
    }

    fn read_file(file: &str) -> Self {
        let file_name = Path::new(file).file_name().unwrap().to_str().unwrap();
        let (id, kind) = SampleID::parse_file_name(file_name);
        let data = match kind {
            "fits" => Some(DataFormat::Fits(Fits::from_file(file))),
            "csv" => Some(DataFormat::Csv(Csv::from_file(file))),
            "temp" => Some(DataFormat::Temp(Temp::from_file(file))),
            _ => None,
        };
        DataFile::new(id, data)
    }
}

impl SampleID {
    fn new(name: String, scan_id: u32, file_id: u32) -> Self {
        SampleID { name, scan_id, file_id }
    }

    fn parse_file_name(file_name: &str) -> (Self, &str) {
        if file_name.contains("Captured Image") {
            let parts: Vec<&str> = file_name.split("Captured Image ").collect();
            return (
                Self {
                    name: String::from("Capture"),
                    scan_id: parts[1].split(" ").collect::<Vec<&str>>()[1].parse::<u32>().unwrap(),
                    file_id: 0,
                },
                "temp",
            );
        } else {
            let re = Regex::new(r"([0-9]{5})-(\d+)\.fits$").unwrap();
            if let Some(caps) = re.captures(file_name) {
                let scan_id = caps.get(1).unwrap().as_str().to_string();
                let file_id = caps.get(2).unwrap().as_str().to_string();
                let sample_name = file_name.split(scan_id.as_str()).collect::<Vec<&str>>()[0]
                    .trim_end_matches('-')
                    .to_string();
                return (
                    Self {
                        name: sample_name,
                        scan_id: scan_id.parse().unwrap(),
                        file_id: file_id.parse().unwrap(),
                    },
                    "fits",
                );
            } else {
                panic!("Invalid file: {}", file_name);
            }
        }
    }
}

impl Fits {
    fn new(pol: Pol, energy: f64, current: f64, hos: f64, hes: f64, exposure: f64, theta: f64, image: io::hdus::image::image::ImageData) -> Self {
        Fits { pol, energy, current, hos, hes, exposure, theta , image }
    }

    fn from_file(file: &str) -> Self {
        let mut fits_file = File::open(file).unwrap();
        let mut header = io::Header::new();
        header.read_from_file(&mut fits_file)?;

        let image_hdu = io::hdus::image::image::ImageParser::read_from_buffer(&mut fits_file, &mut header)?;

        let pol_card = match header.get_card("EPU Polarization") {
            Some(card) => card.value.to_string(),
            None => String::from("N/A"),
        };
        let pol_state = match pol_card.as_str() {
            "100" => Pol::S,
            "190" => Pol::P,
            _ => Pol::N,
        };
        let energy = header.get_card("Beamline Energy").map_or(0.0, |card| card.value.as_float().unwrap_or(0.0));
        let theta = header.get_card("Sample Theta").map_or(0.0, |card| card.value.as_float().unwrap_or(0.0));
        let current = header.get_card("Beam Current").map_or(0.0, |card| card.value.as_float().unwrap_or(0.0));
        let exposure = header.get_card("EXPOSURE").map_or(0.0, |card| card.value.as_float().unwrap_or(0.0));
        let hos = header.get_card("Higher Order Suppressor").map_or(0.0, |card| card.value.as_float().unwrap_or(0.0));
        let hes = header.get_card("Horizontal Exit Slit Size").map_or(0.0, |card| card.value.as_float().unwrap_or(0.0));

        let image = image_hdu.data;

        Fits::new(pol_state, energy, current, hos, hes, exposure, theta, image)
    }
}

impl Csv {
    fn new() -> Self {
        Csv {}
    }

    fn from_file(_file: &str) -> Self {
        Csv::new()
    }
}

impl Temp {
    fn new() -> Self {
        Temp {}
    }

    fn from_file(_file: &str) -> Self {
        Temp::new()
    }
}


Error: the `?` operator can only be used in a method that returns `Result` or `Option` (or another type that implements `FromResidual`)

Error: the `?` operator can only be used in a method that returns `Result` or `Option` (or another type that implements `FromResidual`)

Error: no field `data` on type `ImageData`

In [42]:
let file = r"C:\Users\hduva\pyref\src\refl\ZnPc82862-00001.fits";
let out = DataFile::read_file(file);

In [43]:
println!("{:#?}", out);

DataFile {
    id: SampleID {
        name: "ZnPc",
        scan_id: 82862,
        file_id: 1,
    },
    data: Some(
        Fits(
            Fits {
                pol: S,
                energy: 283.702099502383,
                current: 499.725799560547,
                hos: 8.79951333595377,
                hes: 3000.0,
                exposure: 0.00100000004749745,
                theta: 0.0,
                image: FitsData::U8([[190, 11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82, 190],
                 [11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82, 190, 11, 242, 82,